# 爬取图片

In [4]:
# -*- coding: utf-8 -*-
import random
import csv
import json
import time
import requests
from bs4 import BeautifulSoup
x = random.randint(3, 7)

request_headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-CN,zh-Hans;q=0.9",
    "Connection": "keep-alive",
    "Host": "www.xiaohongshu.com",
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Mobile/15E148 Safari/604.1"
}

# cursor='646e10050000000013037717'

# 失败重试机制, 重试次数为参数
def retry(func, retry_times=3):
    def wrapper(*args, **kwargs):
        for i in range(retry_times):
            try:
                result = func(*args, **kwargs)
                return result
            except Exception as e:
                print(f'第{i+1}次请求失败, {e}')
                time.sleep(1)
    return wrapper

def get_note_details(node_id):
    noteUrl = f'https://www.xiaohongshu.com/explore/{node_id}'
    response = requests.get(noteUrl, headers=request_headers, timeout=10)
    response.encoding = response.apparent_encoding

    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    json_str = soup.find(attrs={'type': 'application/ld+json'}).text

    # 去除换行 等字符 不然 json无法解析 ,Windows 正常 ，Linux 异常
    json_str = json_str.replace('\n', '').replace('\r\n', '')
    result = json.loads(json_str, strict=False)
    if result['name'] != '':
#         print(result)
        return result
    else:
        raise Exception('获取详情失败：' + str(response))

def get_page(page_id, cursor='648cf1200000000013012272', sort_by='hot'):
    # 请求列表
    url = 'https://www.xiaohongshu.com/web_api/sns/v3/page/notes'
    params = {
        'page_size': 20,
        'sort': sort_by,
        'page_id': page_id,
        'cursor': cursor,
        'sid': ''
    }
    headers = {
        'authority': 'www.xiaohongshu.com',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'zh,en;q=0.9',
        'cache-control': 'no-cache',
        'cookie': 'smidV2=202204232153470815582356eb14689cc3de5c1e9509c700d854701c3e3a830; gid.sig=J51gqQITVoxM93_so_lXzHqIjeEsRkRzwnpCG-HeHF4; gid.sign.sig=c3QsgEsA_7IIMWY5_glVUzb3NYboY2AOn4vAAIAyu28; gid.ss=gSMQ9UOnDuZwH2oRGJG6BW6e4grs67TaYpnrW+8Wmd2azBbPYqKXIdsuljVz7UBg; timestamp2=1673265619526e4926c09daf76250bf9aa1ed5c6196f6faa2de07c82866f657; timestamp2.sig=O7AvEb24cf7yh5PwyqhN_au9q62nBL_3BNgT9Ff3504; xhsTrackerId=2fc58f61-b4e6-4736-b3c3-696fcbe0ef62; xhsTrackerId.sig=H52BZu6eM9xIkZTwMYc8r8Jp282ITn1Oa7Y2c41H2B8; a1=18612f30083g3ag1rrkgq76869imyxeub8hc88kz730000246697; webId=14e17cb02fbef4963ab87952fd4e3b45; gid=yYKyJiq8JiD0yYKyJiq88YYTYqkq0kyFFTkAxuWWKYKj3Sq86vCd7Y888J4KKjW8iKjD242j; gid.sign=oB2LoOOCblZWcXwhc2Vs2OyJQNU=; xhsTracker=url=explore&searchengine=google; xhsTracker.sig=MRMGzEloSNQs_DaToApEc',
        'pragma': 'no-cache',
        'referer': 'https://www.xiaohongshu.com/explore',
        'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 16_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Mobile/15E148 Safari/604.1'
    }

    response = requests.get(url, params=params, headers=headers)

    try:
        response.raise_for_status()
        notes = response.json()['data']['notes']
        next_cursor = response.json()['data']['cursor']
        has_more = response.json()['data']['has_more']
        return {'notes': notes, 'next_cursor': next_cursor, 'has_more': has_more}
    except:
        raise Exception('请求接口失败')

def get_notes(page_id, file_name, min_size=1000, sort_by='hot'):
    # 获取笔记列表
    has_more = True
    next_cursor = ''
    success_count = 0
    failed_count = 0
    page_count = 0
    with open(f'downloads/{file_name}.csv', 'a', newline='', encoding='utf-8-sig') as csvfile:
        fieldnames = ['id', 'title', 'content', 'likes', 'collects', 'date', 'image']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        while has_more and success_count < min_size:
            page_count += 1
            list_result = retry(get_page, 3)(page_id, next_cursor, sort_by)
            notes = list_result['notes']
            next_cursor = list_result['next_cursor']
            if next_cursor is not None:
                has_more = list_result['has_more']

            print(f'第{page_count}页')
#             data = []
            for note in notes:
                if note['type'] != 'video':
                    # 加入 sleep 时间
                    time.sleep(x)
                    detail = retry(get_note_details, 3)(note['id'])
                    if detail is not None:
                        # 加入 sleep 时间
                        time.sleep(x)
                        success_count += 1
                        title = detail['name']
                        content = detail['description']
                        date = detail['datePublished']
                        likes = note['likes']
                        collects = note['collects']
                        # convert image data to json string
                        image_json = json.dumps(detail['image'])                               
                        # add note data to list
                        # 存储至 CSV 文件中
                        writer.writerow({'id': note['id'], 'title': title, 'content': content, 'likes': likes, 'collects': collects, 'date': date, 'image': image_json})
                        print(f'成功{success_count}条')
                    else:
                        failed_count += 1
                        print('爬取失败: ' + note['id'])
    print(f'爬取完成，共{success_count}条，失败{failed_count}条')

get_notes('5d2d51683ae0ba0001033e12', 'update', 1000, 'time')


第1页
成功1条
成功2条
成功3条
成功4条
成功5条
成功6条
成功7条
成功8条
成功9条
成功10条
成功11条
成功12条
成功13条
成功14条
成功15条
成功16条
第2页
成功17条
成功18条
成功19条
成功20条
成功21条
成功22条
成功23条
成功24条
成功25条
成功26条
成功27条
成功28条
成功29条
成功30条
成功31条
第3页
成功32条
成功33条
成功34条
成功35条
成功36条
成功37条
成功38条
成功39条
成功40条
成功41条
成功42条
成功43条
成功44条
成功45条
成功46条
成功47条
成功48条
第4页
成功49条
成功50条
成功51条
成功52条
成功53条
成功54条
成功55条
成功56条
成功57条
成功58条
成功59条
成功60条
成功61条
成功62条
成功63条
第5页
成功64条
成功65条
成功66条
成功67条
成功68条
成功69条
成功70条
成功71条
成功72条
成功73条
成功74条
成功75条
成功76条
成功77条
成功78条
成功79条
第6页
成功80条
成功81条
成功82条
成功83条
成功84条
成功85条
成功86条
成功87条
成功88条
成功89条
成功90条
成功91条
成功92条
成功93条
成功94条
成功95条
第7页
成功96条
成功97条
成功98条
成功99条
成功100条
成功101条
成功102条
成功103条
成功104条
成功105条
成功106条
成功107条
成功108条
成功109条
成功110条
成功111条
成功112条
成功113条
第8页
成功114条
成功115条
成功116条
成功117条
成功118条
成功119条
成功120条
成功121条
成功122条
成功123条
成功124条
成功125条
成功126条
成功127条
成功128条
第9页
成功129条
成功130条
成功131条
成功132条
成功133条
成功134条
成功135条
成功136条
成功137条
成功138条
成功139条
成功140条
成功141条
成功142条
成功143条
成功144条
第10页
成功145条
成功146条
成功147条
成功148条
成功149条
成功150条
成功151条
成功152条
成功1

KeyboardInterrupt: 

# 爬取视频 

In [3]:
import random
import csv
import json
import time
import requests
from bs4 import BeautifulSoup

x = random.randint(3, 7)

request_headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-CN,zh-Hans;q=0.9",
    "Connection": "keep-alive",
    "Host": "www.xiaohongshu.com",
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Mobile/15E148 Safari/604.1"
}

# 失败重试机制, 重试次数为参数
def retry(func, retry_times=3):
    def wrapper(*args, **kwargs):
        for i in range(retry_times):
            try:
                result = func(*args, **kwargs)
                return result
            except Exception as e:
                print(f'第{i+1}次请求失败, {e}')
                time.sleep(1)
    return wrapper

def get_note_details(node_id):
    noteUrl = f'https://www.xiaohongshu.com/explore/{node_id}'
    response = requests.get(noteUrl, headers=request_headers, timeout=10)
    response.encoding = response.apparent_encoding

    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    json_str = soup.find(attrs={'type': 'application/ld+json'}).text

    # 去除换行和空格等不必要的字符
    json_str = json_str.replace('\n', '').replace(' ', '')
    result = json.loads(json_str, strict=False)

    if result['name'] != '' and result['@type'] == 'VideoObject':
        return node_id

def get_page(page_id, cursor='648cf1200000000013012272', sort_by='hot'):
    # 请求列表
    url = 'https://www.xiaohongshu.com/web_api/sns/v3/page/notes'
    params = {
        'page_size': 20,
        'sort': sort_by,
        'page_id': page_id,
        'cursor': cursor,
        'sid': ''
    }
    headers = {
        'authority': 'www.xiaohongshu.com',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'zh,en;q=0.9',
        'cache-control': 'no-cache',
        'cookie': 'smidV2=202204232153470815582356eb14689cc3de5c1e9509c700d854701c3e3a830; gid.sig=J51gqQITVoxM93_so_lXzHqIjeEsRkRzwnpCG-HeHF4; gid.sign.sig=c3QsgEsA_7IIMWY5_glVUzb3NYboY2AOn4vAAIAyu28; gid.ss=gSMQ9UOnDuZwH2oRGJG6BW6e4grs67TaYpnrW+8Wmd2azBbPYqKXIdsuljVz7UBg; timestamp2=1673265619526e4926c09daf76250bf9aa1ed5c6196f6faa2de07c82866f657; timestamp2.sig=O7AvEb24cf7yh5PwyqhN_au9q62nBL_3BNgT9Ff3504; xhsTrackerId=2fc58f61-b4e6-4736-b3c3-696fcbe0ef62; xhsTrackerId.sig=H52BZu6eM9xIkZTwMYc8r8Jp282ITn1Oa7Y2c41H2B8; a1=18612f30083g3ag1rrkgq76869imyxeub8hc88kz730000246697; webId=14e17cb02fbef4963ab87952fd4e3b45; gid=yYKyJiq8JiD0yYKyJiq88YYTYqkq0kyFFTkAxuWWKYKj3Sq86vCd7Y888J4KKjW8iKjD242j; gid.sign=oB2LoOOCblZWcXwhc2Vs2OyJQNU=; xhsTracker=url=explore&searchengine=google; xhsTracker.sig=MRMGzEloSNQs_DaToApEc',
        'pragma': 'no-cache',
        'referer': 'https://www.xiaohongshu.com/explore',
        'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 16_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Mobile/15E148 Safari/604.1'
    }

    response = requests.get(url, params=params, headers=headers)

    try:
        response.raise_for_status()
        notes = response.json()['data']['notes']
        next_cursor = response.json()['data']['cursor']
        has_more = response.json()['data']['has_more']
        return {'notes': notes, 'next_cursor': next_cursor, 'has_more': has_more}
    except:
        raise Exception('请求接口失败')

def get_video_note_ids(page_id, file_name, min_size=1000, sort_by='hot'):
    # 获取笔记列表
    has_more = True
    next_cursor = ''
    success_count = 0
    failed_count = 0
    page_count = 0
    video_note_ids = []

    while has_more and success_count < min_size:
        page_count += 1
        list_result = retry(get_page, 3)(page_id, next_cursor, sort_by)
        notes = list_result['notes']
        next_cursor = list_result['next_cursor']
        if next_cursor is not None:
            has_more = list_result['has_more']

        print(f'第{page_count}页')

        for note in notes:
            if note['type'] == 'video':
                time.sleep(x)
                video_note_id = retry(get_note_details, 3)(note['id'])
                if video_note_id is not None:
                    success_count += 1
                    video_note_ids.append(video_note_id)
                    print(f'成功{success_count}条')
                else:
                    failed_count += 1
                    print('获取失败: ' + note['id'])
    
    # 将视频note id保存到文件中
    with open(f'{file_name}.txt', 'w') as file:
        for note_id in video_note_ids:
            file.write(note_id + '\n')
    
    print(f'获取视频note ID完成，共{success_count}条，失败{failed_count}条')

get_video_note_ids('5d2d51683ae0ba0001033e12', 'video_note_ids', 1000, 'time')

第1页
成功1条
成功2条
第2页
成功3条
成功4条
成功5条
成功6条
成功7条
第3页
成功8条
第4页
成功9条
成功10条
第5页
成功11条
成功12条
第6页
成功13条
成功14条
成功15条
第7页
第8页
成功16条
成功17条
成功18条
成功19条
第9页
成功20条
成功21条
第10页
成功22条
成功23条
成功24条
成功25条
成功26条
第11页
成功27条
成功28条
成功29条
成功30条
成功31条
第12页
成功32条
成功33条
第13页
第14页
成功34条
成功35条
成功36条
成功37条
成功38条
成功39条
第15页
第16页
成功40条
成功41条
成功42条
成功43条
成功44条
成功45条
成功46条
第17页
成功47条
成功48条
成功49条
成功50条
第18页
成功51条
成功52条
成功53条
成功54条
第19页
成功55条
成功56条
第20页
成功57条
成功58条
成功59条
第21页
成功60条
第22页
成功61条
成功62条
成功63条
成功64条
第23页
成功65条
成功66条
成功67条
成功68条
第24页
成功69条
成功70条
成功71条
成功72条
第25页
成功73条
成功74条
成功75条
成功76条
第26页
成功77条
成功78条
成功79条
成功80条
第27页
成功81条
成功82条
成功83条
成功84条
成功85条
成功86条
第28页
成功87条
成功88条
成功89条
成功90条
成功91条
第29页
成功92条
成功93条
成功94条
成功95条
成功96条
第30页
成功97条
第31页
成功98条
成功99条
成功100条
第32页
成功101条
成功102条
成功103条
成功104条
第33页
成功105条
成功106条
成功107条
第34页
成功108条
第35页
成功109条
成功110条
成功111条
成功112条
第36页
成功113条
成功114条
成功115条
成功116条
第37页
成功117条
成功118条
成功119条
成功120条
成功121条
成功122条
成功123条
第38页
第39页
成功124条
成功125条
成功126条
第40页
第41页
成功127条
成功128条
成功129条
第42页
成功13